In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv("Documents/Materials.csv",encoding = "ISO-8859-1")
df.head()

,Unnamed: 0,DEPT,STORE_CODE,PREFIX_MTL_CODE,MTL_CODE,LOCATION_CODE,MTL_DESC,MAT_DESC,UNIT,CURRENT_RATE
0,1,206,2,TR,4670263510,M/SHED R-8 KE SAMNE,CU DET,CU DET,NO,7138.60
1,2,209,3,SG,4900107255,KA-40,RELAY VAGM,RELAY VAGM,NO,5420.33
2,3,209,3,SG,4900107280,MR,RELAY VAJH-13,RELAY VAJH-13,NO,1300.93
3,4,209,3,SG,4900107298,KB-48,RELAY VAX31,RELAY VAX31,NO,4428.94
4,5,209,3,SG,4900107301,MR,RELAY DTH31,RELAY DTH31,NO,36412.03


In [3]:
df.shape

(193379, 10)

In [4]:
features=["MAT_DESC","UNIT","MTL_CODE","CURRENT_RATE"]
df1=df[features]
df[features].isnull().sum()

MAT_DESC        1669
UNIT               1
MTL_CODE           0
CURRENT_RATE       0
dtype: int64

In [8]:
df1.dropna(axis=0,how='any',inplace=True)

C:\Users\6031811test.BHELBPL\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
df1.isnull().sum()

MAT_DESC        0
UNIT            0
MTL_CODE        0
CURRENT_RATE    0
dtype: int64

In [12]:
df2=df1[:1000]

In [13]:
from collections import Counter
all_names = df2['MAT_DESC'][df2['CURRENT_RATE']>-19]
names_freq = Counter()
for name in all_names:
    names_freq.update(str(name).split(" /"))
key_words = [word for (word,_) in names_freq.most_common(30)]
print(key_words)

['PIN', 'WASHER', 'LABEL', 'LED LAMP', 'TERMINAL', 'CT', 'SPRING', 'LEVER', 'BRACKET', 'CU DET', 'SPACER', 'METER', 'RELAY', 'GASKET', 'PT', 'SLEEVE', 'SHIM', 'NUMBER DISC', 'CLIP', 'AMMETER', 'SUPPORT ROLLER', 'BUSH', 'ROD', 'SWING-BOLT', 'STUD', 'ROLLER', 'LIMIT SWITCH', 'INDICATING LAMP.', 'TUBE', 'COPPER CABLE']


In [14]:
len(all_names)

1000

In [15]:
all_names=all_names.tolist()

In [16]:
all_main_name = pd.DataFrame(columns=['MTL_CODE','sort_gp','names','alias','score','unit','price'])
all_names.sort()
all_main_name['names'] = all_names
all_main_name['sort_gp'] = all_main_name['names'].apply(lambda x: x[0])
all_main_name['MTL_CODE'] =df2['MTL_CODE']
all_main_name['unit']=df2['UNIT']
all_main_name['price']=df2['CURRENT_RATE']

In [17]:
all_main_name.shape

(1000, 7)

In [18]:
from fuzzywuzzy import fuzz

all_sort_gp = all_main_name['sort_gp'].unique()
n=0

def no_key_word(name):
   
    output = True
    for key in key_words:
        if key in name:
            output = False
    return output

for sortgp in all_sort_gp:
    this_gp = all_main_name.groupby(['sort_gp']).get_group(sortgp)
    gp_start = this_gp.index.min()
    gp_end = this_gp.index.max()
    for i in range(gp_start,gp_end+1):
    
        # if self has not got alias, asign to be alias of itself
        if pd.isna(all_main_name['alias'].iloc[i]):
            all_main_name['alias'].iloc[i] = all_main_name['names'].iloc[i];
            all_main_name['score'].iloc[i] = 100
        
        # if the following has not got alias and fuzzy match, asign to be alias of this one
        for j in range(i+1,gp_end+1):
            if pd.isna(all_main_name['alias'].iloc[j]):
                fuzz_socre = fuzz.token_set_ratio(all_main_name['names'].iloc[i],all_main_name['names'].iloc[j])
                if not no_key_word(all_main_name['names'].iloc[j]):
                    fuzz_socre -= 10
                if (fuzz_socre > 75):
                    all_main_name['alias'].iloc[j] = all_main_name['alias'].iloc[i]
                    all_main_name['score'].iloc[j] = fuzz_socre
                    
        if i % (len(all_names)//10) == 0:
            print("progress: %.2f" % (100*i/len(all_names)) + "%")
                



C:\Users\6031811test.BHELBPL\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
C:\Users\6031811test.BHELBPL\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


progress: 0.00%
progress: 10.00%
progress: 20.00%
progress: 30.00%
progress: 40.00%
progress: 50.00%
progress: 60.00%
progress: 70.00%
progress: 80.00%
progress: 90.00%


In [19]:
all_main_name[:10]

,MTL_CODE,sort_gp,names,alias,score,unit,price
0,4.670264e+09,"""","""DMC"" COMPOUND TO AA22826","""DMC"" COMPOUND TO AA22826",100,NO,7138.60
1,4.900107e+09,','O'RING,'O'RING,100,NO,5420.33
2,4.900107e+09,','YASHMUN' INDUSTRIAL TYPE BELL UNITS,'YASHMUN' INDUSTRIAL TYPE BELL UNITS,100,NO,1300.93
3,4.900107e+09,0,0-1500 OHMS RESISTOR,0-1500 OHMS RESISTOR,100,NO,4428.94
4,4.900107e+09,0,0.65 MM STATOR SEGMENT,0.65 MM STATOR SEGMENT,100,NO,36412.03
5,4.900107e+09,1,1 K OHMS RESISTORS,1 K OHMS RESISTORS,100,NO,3923.89
6,4.900107e+09,1,1-PH PT,1-PH PT,100,NO,3685.39
7,4.900107e+09,1,1.5MMSQ GREEN CABLE,1.5MMSQ GREEN CABLE,100,NO,263.41
8,4.900067e+09,1,1.5X6 PICC-LPD 903.5,1.5X6 PICC-LPD 903.5,100,NO,4858.09
9,4.900067e+09,1,10 MM DIA BRT.STEEL BAR.,10 MM DIA BRT.STEEL BAR.,100,NO,4944.46
